In [2]:
from langchain_groq import ChatGroq
from langchain_openai import AzureChatOpenAI
import os 
# llm=ChatGroq(
#     groq_api_key=groq_api_key,
#     model_name='Gemma2-9b-it'
#     )
llm=AzureChatOpenAI(
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_MODEL'),
    api_version=os.getenv('AZURE_OpenAI_API_VERSION'),
    temperature=0.1,

)
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000279445A91C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000279445D3340>, root_client=<openai.lib.azure.AzureOpenAI object at 0x00000279422C3970>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x00000279445A9220>, temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://easttest123.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-03-15-preview', openai_api_version='2024-02-15-preview', openai_api_type='azure')

In [10]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=os.getenv('NEO4J_URI_ONLINE'),
                 username=os.getenv('NEO4J_USERNAME_ONLINE'),
                 password=os.getenv('NEO4J_PASSWORD_ONLINE')
                #  database=os.getenv('NEO4J_DATABASE')
                 )

In [8]:
import pandas as pd

graph_data=pd.read_csv('./data/cureus/graph.csv',sep='|')

In [9]:
graph_data 

,node_1,node_2,edge,chunk_id
0,mark zuckerberg,wikipedia,Mark Zuckerberg is mentioned in the Wikipedia ...,c907bd16ca9e42c291e3f3c00f6b7670
1,mark zuckerberg,facebook,Mark Zuckerberg is associated with Facebook.,c907bd16ca9e42c291e3f3c00f6b7670
2,mark zuckerberg,chan zuckerberg initiative,Mark Zuckerberg is involved with the Chan Zuck...,c907bd16ca9e42c291e3f3c00f6b7670
3,mark zuckerberg,the social network,Mark Zuckerberg is depicted in the movie 'The ...,c907bd16ca9e42c291e3f3c00f6b7670
4,mark zuckerberg,marriage and children,Mark Zuckerberg has a marriage and children.,c907bd16ca9e42c291e3f3c00f6b7670
...,...,...,...,...
325,terms of use,privacy policy,Terms of Use and Privacy Policy are legal docu...,849352f02f9043b5a14b80b0eda2cfe5
326,wikipedia,wikimedia foundation inc,Wikipedia is a registered trademark owned by t...,849352f02f9043b5a14b80b0eda2cfe5
327,wikipedia,privacy policy,Wikipedia has its own Privacy Policy that outl...,849352f02f9043b5a14b80b0eda2cfe5
328,wikipedia,about wikipedia,About Wikipedia provides information about the...,849352f02f9043b5a14b80b0eda2cfe5


In [11]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(os.getenv('NEO4J_URI_ONLINE'),auth=(os.getenv('NEO4J_USERNAME_ONLINE'),os.getenv('NEO4J_PASSWORD_ONLINE')))
def insert_data_to_neo4j(dataframe):
    with driver.session() as session:
        for _,row in dataframe.iterrows():
            node1=row['node_1']
            node2=row['node_2']
            edge_description=row['edge']

            query=""" 
                MERGE (n1:Entity {name: $node1})
                MERGE (n2:Entity {name: $node2})
                MERGE (n1)-[:RELATION {description: $edge_description}]->(n2)
            """

            session.run(query,node1=node1,node2=node2,edge_description=edge_description)
insert_data_to_neo4j(graph_data)

In [12]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000027949BB68E0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000279445A91C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000279445D3340>, 

In [15]:
response=chain.invoke({'query':'when did he start facebook'})
response



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: FOUNDED)} {position: line: 1, column: 21, offset: 20} for query: "MATCH (p:Person)-[r:FOUNDED]->(c:Company {name: 'Facebook'})\nRETURN r.start_date"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (

Generated Cypher:
MATCH (p:Person)-[r:FOUNDED]->(c:Company {name: 'Facebook'})
RETURN r.start_date
Full Context:
[]

> Finished chain.


{'query': 'when did he start facebook',
 'result': "I don't know the answer to that question."}